# Testes de Hipóteses

In [55]:
import pandas as pd

In [84]:
# Load gun violence dataset and gun laws dataset
gun_violence = pd.read_csv('../databases/gun_violence_clean.zip', compression='zip')
gun_law = pd.read_csv('../databases/gun_laws.zip', compression='zip')
population = pd.read_csv('../databases/population_clean.zip', compression='zip')

## Teste 1

Hipotese nula: dado que o Estado apresenta um maior número de leis que dizem respeito ao controle de porte de armas, ele apresentaria uma porcentagem de incidentes menor do que estados de menos leis.

In [87]:
gun_violence = gun_violence[['incident_id', 'state', 'n_killed', 'n_injured', 'year', 'month', 'day']]
gun_violence = gun_violence[~gun_violence['year'].isin(['2013', '2018'])]
glaw = gun_law.iloc[:, 1:]
population = population[['2014', '2015', '2016', '2017']]

pop = population[5:56].reset_index().drop('index', axis=1)

incidents_state = gun_violence.groupby('state')\
                  .agg({'n_injured':'sum', 'incident_id':'count', 'n_killed':'sum'})\
                  .rename(columns={'incident_id':'number_of_incidents'})\
                  .sort_values('state')\
                  .reset_index()
states = pd.concat([incidents_state, pop, glaw], axis=1, sort=False, join='inner')
states['mean_pop'] = ((states['2014'] + states['2015'] + states['2016'] +states['2017'])/4)
states['porcent_killed'] = states['n_killed']/ states['mean_pop']
states['porcent_incidents'] = states['number_of_incidents']/ states['mean_pop']
states['porcent_injured'] = states['n_injured']/ states['mean_pop']
states

,state,n_injured,number_of_incidents,n_killed,2014,2015,2016,2017,Red Flag Laws,Relinquishment Laws,Assault Weapon Ban,High-Capacity Magazine ban,Prohibition for high-risk individuals,Prohibitions for individual with domestic violence covictions,Mandatory universal background checks,mean_pop,porcent_killed,porcent_incidents,porcent_injured
0,Alabama,588,1353,757,4842481,4853160,4864745,4875120,0,0,0,0,1,1,0,4858876.50,0.000156,0.000278,0.000121
1,Alaska,95,523,139,736307,737547,741504,739786,0,0,0,0,1,0,0,738786.00,0.000188,0.000708,0.000129
2,Arizona,304,742,524,6733840,6833596,6945452,7048876,0,0,0,0,1,1,0,6890441.00,0.000076,0.000108,0.000044
3,Arkansas,479,934,346,2967726,2978407,2990410,3002997,0,0,0,0,1,0,0,2984885.00,0.000116,0.000313,0.000160
4,California,1796,5032,2143,38625139,38953142,39209127,39399349,1,1,1,1,1,1,1,39046689.25,0.000055,0.000129,0.000046
5,Colorado,243,777,321,5351218,5452107,5540921,5615902,0,0,0,1,1,1,1,5490037.00,0.000058,0.000142,0.000044
6,Connecticut,331,1086,144,3594783,3587509,3578674,3573880,1,1,1,1,1,1,1,3583711.50,0.000040,0.000303,0.000092
7,Delaware,331,621,78,932596,941413,949216,957078,0,0,0,0,1,1,1,945075.75,0.000083,0.000657,0.000350
8,District of Columbia,129,277,150,662513,675254,686575,695691,0,0,1,1,1,1,1,680008.25,0.000221,0.000407,0.000190
9,Florida,2015,4396,1566,19860330,20224249,20629982,20976812,0,0,0,0,1,1,0,20422843.25,0.000077,0.000215,0.000099


## Teste 2